In [11]:
import boto3
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.llms.bedrock import Bedrock
from langchain.document_loaders import S3FileLoader
from bedrock import get_bedrock_client
bedrock_client = get_bedrock_client(region='us-east-1', runtime=True)

s3 = boto3.client('s3')

bucket_name = 'my-s3-doc-loader'
response = s3.list_objects_v2(Bucket=bucket_name)

file_names = []
for obj in response['Contents']:
    key = obj['Key']
    file_names.append(key)

for context_key in file_names:
    loader = S3FileLoader(bucket=bucket_name, key=context_key)
    context_content = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000,separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""])
context_texts = text_splitter.split_documents(context_content)

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = FAISS.from_documents(documents=context_texts, embedding=embeddings)

retriever = db.as_retriever(search_type='mmr', search_kwargs={"k": 5})

template = """
Human: Answer truthfully based on the given question, fetch the answer only from the given text documents
Instruction:
1.If multiple files are there, read the all the files each and every lines accurately for to generate answer
2.If there is no text found in the text document about the asked question ,"print no result found"
3.Generate answer whatever available related to the question
4.Must complete the sentence in the result fully, do not leave results incomplete format in the end.
text:{context}
question:{question}
Assistant:"""
qa_prompt = PromptTemplate(template=template, input_variables=["context","question"])
chain_type_kwargs = { "prompt": qa_prompt}
llm = Bedrock(model_id="anthropic.claude-v2",client=bedrock_client)
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs=chain_type_kwargs,
    verbose=False
)


In [22]:
question="what is Glaucoma?"
result = qa.run(question)
print(result)

 Based on the given text documents, here is the answer to your question about glaucoma:

Glaucoma is an eye condition that damages the optic nerve, which is vital for good vision. This damage is often caused by an abnormally high pressure in your eye. Glaucoma tends to develop slowly over time and can lead to loss of vision if it's not treated.

The optic nerve is a bundle of more than 1 million nerve fibers that connect the retina to the brain. It runs from your eye to your brain and is key to good vision. Glaucoma damages the optic nerve, often because of increased pressure in your eye. Over time, glaucoma can diminish peripheral vision.

Glaucoma is one of the leading causes of blindness for people over 60 years old. But blindness from glaucoma can often be prevented with early treatment.

No result found in the given text related to symptoms or treatment of glaucoma.
